# Capstone Project - Car Accident Severity

In [1]:
# importing the required libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

## Preprocessing the dataset

In [2]:
#importing the dataset
!wget -O collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-08-31 16:09:02--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘collisions.csv’

100%[======================================>] 73,917,638  36.0MB/s   in 2.0s   

2020-08-31 16:09:04 (36.0 MB/s) - ‘collisions.csv’ saved [73917638/73917638]



In [3]:
#converting the csv dataset to a pandas datafram for use in machine learning
df = pd.read_csv('collisions.csv')
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [4]:
df.shape

(194673, 38)

In [5]:
#seeing if the severity column is balanced
df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [6]:
#balancing the severity column by downsampling the code 1 cases
df_1 = df[df.SEVERITYCODE == 1]
df_2 = df[df.SEVERITYCODE == 2]
df_1_downsampled = resample(df_1, replace = False, n_samples = 58188, random_state = 123)
df_downsampled = pd.concat([df_1_downsampled, df_2])
df_downsampled.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [7]:
#Creating our feature dataframe for modelling whilst converting it from categorical to numerical data types using label encoding
Feature_df = df_downsampled[['WEATHER', 'ROADCOND', 'LIGHTCOND']]
Feature_df = Feature_df.astype('category')
Feature_df['WEATHER_INT'] = Feature_df['WEATHER'].cat.codes
Feature_df['ROADCOND_INT'] = Feature_df['ROADCOND'].cat.codes
Feature_df['LIGHTCOND_INT'] = Feature_df['LIGHTCOND'].cat.codes
Feature = Feature_df[['WEATHER_INT', 'ROADCOND_INT', 'LIGHTCOND_INT']]
Feature.head()

,WEATHER_INT,ROADCOND_INT,LIGHTCOND_INT
25055,6,8,2
65280,1,0,5
86292,10,7,8
155111,1,0,5
64598,1,0,5


In [8]:
#Defining X and y for modelling
X = Feature
y = df_downsampled['SEVERITYCODE'].values
y[0:5]

array([1, 1, 1, 1, 1])

In [9]:
#normalizing X
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype int8 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


array([[ 1.15236718,  1.52797946, -1.21648407],
       [-0.67488   , -0.67084969,  0.42978835],
       [ 2.61416492,  1.25312582,  2.07606076],
       [-0.67488   , -0.67084969,  0.42978835],
       [-0.67488   , -0.67084969,  0.42978835]])

## K-Nearest Neighbour (KNN)

In [10]:
#Splitting X into training and testing sets to find the best k
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (81463, 3) (81463,)
Test set: (34913, 3) (34913,)


In [11]:
Finding the best value of k 
Ks=30
mean_acc=np.zeros((Ks-1))
std_acc=np.zeros((Ks-1))
ConfustionMx=[];
for n in range(1,Ks):
    
    kNN_model = KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)
    yhat = kNN_model.predict(X_test)
    
    
    mean_acc[n-1]=np.mean(yhat==y_test);
    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

In [12]:
#Training and predicting the model with the optimal k value
k = 25
kNN_model = KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
yhat = kNN_model.predict(X_test)
yhat

array([2, 2, 1, ..., 2, 2, 2])

In [13]:
#Evaluating our model
print("KNN Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("KNN F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

KNN Jaccard index: 0.56
KNN F1-score: 0.54


## Decision Tree

In [14]:
#Training the decision tree model
DT_model = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT_model.fit(X_train,y_train)
DT_model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
yhat = DT_model.predict(X_test)
yhat

array([2, 2, 1, ..., 2, 2, 2])

In [16]:
#Evaluating the accuracy of the model
print("Decision Tree Jaccard index: %.2f" % jaccard_similarity_score(y_test, yhat))
print("Decision Tree F1-score: %.2f" % f1_score(y_test, yhat, average='weighted') )

Decision Tree Jaccard index: 0.56
Decision Tree F1-score: 0.48


## Logistic Regression

In [17]:
LR_model = LogisticRegression(C=0.01).fit(X_train,y_train)
LR_model

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
yhat = LR_model.predict(X_test)
yhat

array([1, 2, 1, ..., 2, 2, 2])

In [19]:
LR_yhat = LR_model.predict(X_test)
LR_yhat_prob = LR_model.predict_proba(X_test)
print("LR Jaccard index: %.2f" % jaccard_similarity_score(y_test, LR_yhat))
print("LR F1-score: %.2f" % f1_score(y_test, LR_yhat, average='weighted') )
print("LR LogLoss: %.2f" % log_loss(y_test, LR_yhat_prob))

LR Jaccard index: 0.53
LR F1-score: 0.51
LR LogLoss: 0.68
